# CS337 - OPERATING SYSTEMS- Project 7 Software Synchronization
#### By: Matthew Bass


### Table of Contents

* [Overview](#over)
* [Setup](#set)
* [Conclusion](#con)



**Project Overview:** <a class="anchor" id="over"></a>

In this project I will, write a small class to implement a `Semaphore` class
using a `counter` and a `condition` variable.  Here’s the documentation for
[condition variables/objects in Python](https://docs.python.org/3/library/threading.html#condition-objects)



### PREREQUISITES:
    Python 3
    Jupyter
    jupyter_contrib_nbextensions
    random
    threading
    datclasses
    autopep8


<br>

-----

### Setup: <a class="anchor" id="set"></a>

To start I am using this code to make all my code blocks hide-able, thanks to
[stackoverflow](https://stackoverflow.com/questions/27934885/how-to-hide-code-from-cells-in-ipython-notebook-visualized-with-nbviewer).



In [1]:
# %%HTML
# <script>
#     function luc21893_refresh_cell(cell) {
#         if( cell.luc21893 ) return;
#         cell.luc21893 = true;
#         console.debug('New code cell found...' );
#
#         var div = document.createElement('DIV');
#         cell.parentNode.insertBefore( div, cell.nextSibling );
#         div.style.textAlign = 'right';
#         var a = document.createElement('A');
#         div.appendChild(a);
#         a.href='#'
#         a.luc21893 = cell;
#         a.setAttribute( 'onclick', "luc21893_toggle(this); return false;" );
#
#         cell.style.visibility='hidden';
#         cell.style.position='absolute';
#         a.innerHTML = '[show code]';
#
#     }
#     function luc21893_refresh() {
#         if( document.querySelector('.code_cell .input') == null ) {
#             // it apeears that I am in a exported html
#             // hide this code
#             var codeCells = document.querySelectorAll('.jp-InputArea')
#             codeCells[0].style.visibility = 'hidden';
#             codeCells[0].style.position = 'absolute';
#             for( var i = 1; i < codeCells.length; i++ ) {
#                 luc21893_refresh_cell(codeCells[i].parentNode)
#             }
#             window.onload = luc21893_refresh;
#         }
#         else {
#             // it apperas that I am in a jupyter editor
#             var codeCells = document.querySelectorAll('.code_cell .input')
#             for( var i = 0; i < codeCells.length; i++ ) {
#                 luc21893_refresh_cell(codeCells[i])
#             }
#             window.setTimeout( luc21893_refresh, 1000 )
#         }
#     }
#
#     function luc21893_toggle(a) {
#         if( a.luc21893.style.visibility=='hidden' ) {
#             a.luc21893.style.visibility='visible';
#             a.luc21893.style.position='';
#             a.innerHTML = '[hide code]';
#         }
#         else {
#             a.luc21893.style.visibility='hidden';
#             a.luc21893.style.position='absolute';
#             a.innerHTML = '[show code]';
#         }
#     }
#
#     luc21893_refresh()
# </script>

#### Imports:

In [1]:
import threading
import random
import time
import numpy as np
from dataclasses import dataclass, field



### Semaphore Class:

In [2]:
@dataclass
class Semaphore():
    '''
    A Semaphore class that allows for the use of a counter and a condition
    variable.

    Attributes:
        counter: An integer that represents the number of threads that are
            currently waiting on the condition variable.
        condition: A condition variable that is used to signal threads that
            the counter is 0.
    '''
    counter: int = field(default=1)
    condition: threading.Condition = field(default = threading.Condition(),
                                           init=False)

    def acquire(self):
        '''
        This method acquires the lock for the condition variable before
        decrementing the counter by one, then it checks if the counter is
        below zero and sets the thread to sleep if true.  Otherwise, it
        releases the lock.

        '''

        # acquire the lock for the condition variable with the context manager
        with self.condition:
            # decrement the counter
            self.counter -= 1

            # check if the counter is below zero
            # if so, set the thread to sleep
            # otherwise, release the lock
            if self.counter < 0:
                self.condition.wait()

        return




    def release(self):
        '''
        This method acquires the condition lock and increments the counter by
        one, notifies a single sleeping thread, and releases the lock.
        '''

        # # acquire the lock
        # self.condition.acquire()
        #
        # # increment the counter
        # self.counter += 1
        #
        # # notify a single sleeping thread
        # self.condition.notify()
        #
        # # release the lock
        # self.condition.release()

        # acquire the lock for the condition variable with the context manager
        with self.condition:
            # increment the counter
            self.counter += 1

            # notify a single sleeping thread
            self.condition.notify()

        return

    # Make the Semaphore class a context manager
    __enter__ = acquire
    __exit__ = release


### Buffer Simulation ( Semaphore Test ):

Now to test my `Semaphore` class, I will use the Buffer example from class where
 we used Python semaphores and replace the Python semaphores with my own
 `Semaphore` class. The output should be similar to the python semaphore
 solution, in the sense that:

1. We see no errors, especially pop with empty list error.

2. The buffer never exceeds 10.

3. Every item added to the buffer is removed at the end of each run.


This buffer simulation is a producer and consumer problem.  The producers
will put items into the `buffer`, and the consumers will take items out of the
`buffer`.  The buffer is bounded by a maximum size of 10.  The items will be
unique numbers 1 - `buffer_size`.

There will be 3 `Semaphores` used in this simulation.  The first is
`access` which is used to synchronize access to the `buffer` between the
consumer and producer threads, it starts with a value of 1 for the counter
and is decremented by 1 for each thread that wants to access the buffer,
since only one thread can access the buffer at a time. The
second is `empty` which is used to signal the number of empty spaces in the
`buffer` it starts with a value the size of `buffer_size` and is decremented
by 1 for each thread that adds an item to the `buffer`. The third is `full`
which is used to signal if the `buffer` is full or not and starts
at 0 and is incremented by 1 every time the `buffer` is full.

Before I make the `bufferSimulation()` function, I will define the `Producer` and
`Consumer` threads.

#### Producer Thread:

The `Producer` thead will put unique numbers into the buffer. It inherits from
 the threading.Thread class.


The attributes of the `Producer` class are:
- `producer_id`: An integer that represents the id of the producer.

- `num_produce`: An integer that represents the number of items to produce.

- `buffer`: A list that represents the buffer (shared resource).

- `access`: A `Semaphore` object that represents the number of threads that can
    access the buffer at a time (starts at 1 to represent only one thread can
    access the buffer at a time).

- `empty`: A `Semaphore` object that represents the number of empty spaces in the
    buffer.

- `full`: A `Semaphore` object that represents if the buffer is full or not
(starts at 0 to represent not full and is incremented by 1 every time the buffer
is full).

- `sleep_amt`: An integer that represents the amount of time the thread will
    sleep for after producing an item.

- `debug`: A boolean that represents if the thread should print debug messages
    or not.


This class will produce numbers `(producer_id * num_produce) -
 num_produce + (producer_id * num_produce)` and put them into the buffer
 `num_produce` times.  It will sleep for `sleep_amt` seconds after each item is
 produced.


Here is the code for the `Producer` class:

In [3]:
class Producer(threading.Thread):
    def __init__(self,
                 producer_id: int ,
                 num_produce: int,
                 buffer: list,
                 access: Semaphore,
                 empty: Semaphore,
                 full: Semaphore,
                 sleep_amt: float = 1,
                 debug: bool = False):
        '''
        The constructor for the Producer class.

        Args:
            producer_id: An integer that represents the producer number.
            num_produce: An integer that represents the number of items that
                each producer will put into the buffer.
            buffer: A list that represents the buffer.
            access: A Semaphore that represents the access lock for the buffer.
            empty: A Semaphore that represents the empty lock for the buffer.
            full: A `Semaphore` object that represents if the buffer is full or not
                (starts at 0 to represent not full and is incremented by 1
                every time the buffer is full).
            sleep_amt: A float that represents the amount of time the thread
            debug: A boolean that represents whether or not the debug mode is
        '''
        threading.Thread.__init__(self)
        self.producer_id = producer_id
        self.num_produce = num_produce
        self.buffer = buffer
        self.access = access
        self.empty = empty
        self.full = full
        self.sleep_amt = sleep_amt
        self.debug = debug
        return

    def run(self):
        '''
        The run method for the Producer class.  This method will produce
        numbers 0 + (producer_num * NUM_PRODUCE) - NUM_PRODUCE + (producer_num *
        NUM_PRODUCE) and put them into the buffer.
        '''
        # Make a unique data array
        data_array = np.arange(self.num_produce) + \
                     (self.producer_id * self.num_produce)

        for data in data_array:

            # Acquire the empty semaphore
            self.empty.acquire()

            # Acquire the access semaphore
            self.access.acquire()

            # Add the data to the buffer
            self.buffer.append(data)

            if self.debug:
                print(f"\nProducer {self.producer_id} " +\
                      f"({threading.current_thread().name}) " +\
                      f"added {data} to the buffer. " +\
                      f"Buffer Size: {len(self.buffer)}")


            # Release the access semaphore
            self.access.release()

            # Release the full semaphore
            self.full.release()

            # Sleep for the amount of time specified
            if self.sleep_amt != -1:
                time.sleep(self.sleep_amt)
            else:
                time.sleep((random.random() * 5) + 0.1)

        return

#### Consumer Thread:

The `Consumer` thead will take unique numbers from the buffer. It inherits from
 the threading.Thread class.





The attributes of the `Consumer` class are:
- `consumer_id`: An integer that represents the id of the consumer.

- `num_consume`: An integer that represents the number of items to consume.

- `buffer`: A list that represents the buffer (shared resource).

- `access`: A `Semaphore` object that represents the number of threads that can
    access the buffer at a time (starts at 1 to represent only one thread can
    access the buffer at a time).

- `empty`: A `Semaphore` object that represents the number of empty spaces in the
    buffer.

- `full`: A `Semaphore` object that represents tif the buffer is full or not
(starts at 0 to represent not full and is incremented by 1 every time the buffer
is full).

- `sleep_amt`: An integer that represents the amount of time the thread will
    sleep for after producing an item. if sleep is -1, the thread will sleep for
    a random amount of time.

- `debug`: A boolean that represents if the thread should print debug messages
    or not.


 This is a `Consumer` class that inherits from the threading.Thread class.
    This class will consume `num_produce * (consumer_amt/producer_amt) =
    num_consumed` numbers from the buffer and print them out.


Here is the code for the `Consumer` class:

In [4]:
class Consumer(threading.Thread):

    def __init__(self,
                 consumer_id: int,
                 num_consume: int,
                 buffer: list,
                 access: Semaphore,
                 empty: Semaphore,
                 full: Semaphore,
                 sleep_amt: float = 1,
                 debug: bool = False):
        '''
        The constructor for the Consumer class.

        Args:
            consumer_id: An integer that represents the consumer number.
            num_consume: An integer that represents the number of items that
                each consumer will consume from the buffer.
            buffer: A list that represents the buffer.
            access: A `Semaphore` object that represents the number of
                    threads that can access the buffer at a time (starts at 1
                     to represent only one thread can access the buffer at a
                     time).

            empty: A Semaphore that represents the empty lock for the buffer.
            full: A `Semaphore` object that represents tif the buffer is full or not
                    (starts at 0 to represent not full and is incremented by
                    1 every time the buffer is full).
            sleep_amt: A float that represents the amount of time that the
                consumer will sleep for.
            debug: A boolean that represents whether or not the debug mode is
        '''
        threading.Thread.__init__(self)
        self.consumer_id = consumer_id
        self.num_consume = num_consume
        self.buffer = buffer
        self.access = access
        self.empty = empty
        self.full = full
        self.sleep_amt = sleep_amt
        self.debug = debug
        return

    def run(self):
        '''
        The run method for the Consumer class.  This method will consume
        numbers from the buffer and print them out.
        '''
        for _ in range(self.num_consume):

            # Acquire the full semaphore
            self.full.acquire()

            # Acquire the access semaphore
            self.access.acquire()

            # Remove the data from the buffer
            data = self.buffer.pop(0)

            if self.debug:
                print(f"Consumer {self.consumer_id} " +\
                      f"({threading.current_thread().name}) " +\
                      f"popped {data} from the buffer. " +\
                      f"Buffer Size: {len(self.buffer)}")

            # Release the access semaphore
            self.access.release()

            # Release the empty semaphore
            self.empty.release()

            # Sleep for set sleep amount
            if self.sleep_amt != -1:
                time.sleep(self.sleep_amt)
            else:
                time.sleep((random.random() * 5) + 0.1)


        return


#### Buffer Simulation Function:

 The `bufferSimulation()` function will run the `Producer` and `Consumer`
 threads and the buffer simulation to test the `Semaphore` class.

The `bufferSimulation()` function will take in the following arguments:
- `consumer_amt` (int): The number of `Consumer`s to create.

- `producer_amt` (int): The number of `Producer`s to create.

- `consumer_sleep` (float): The amount of time that each `Consumer` will
sleep. (-1 to represent random amount of time)

- `producer_sleep` (float): The amount of time that each `Producer` will
sleep. (-1 to represent random amount of time)

- `data_amt` (int): The number of items that each `Producer` will produce.

- `buffer_size` (int): The size of the `buffer`.

- `debug` (bool): If true, will print debug statements.




In [5]:
def bufferSimulation(consumer_amt: int = 1, producer_amt: int = 1,
                     consumer_sleep: float = -1, producer_sleep: float = -1,
                     data_amt : int = 20, buffer_size = 10,
                     debug: bool = True):
    '''
    This function will simulate a buffer with a producer and consumers.
    Args:
        consumer_amt (int): The number of consumers to create.
        producer_amt (int): The number of producers to create.
        consumer_sleep (float): The amount of time that each consumer will sleep.
        producer_sleep (float): The amount of time that each producer will sleep.
        data_amt (int): The number of items that each producer will produce.
        buffer_size (int): The size of the buffer.
        debug (bool): If true, will print debug statements.

    Returns:

    '''

    # Make the variables that will be shared among the threads

    buffer = []


    # Semaphores
    access = Semaphore(counter=1)
    empty = Semaphore(counter=buffer_size)
    full = Semaphore(counter=0)

    # Calculate the consumer amount
    num_consume = data_amt // consumer_amt

    # Create the consumers
    consumers = []
    for i in range(consumer_amt):
        consumers.append(Consumer(i, num_consume, buffer, access, empty, full,
                                  consumer_sleep, debug))

    # Create the producers
    producers = []
    for i in range(producer_amt):
        producers.append(Producer(i, data_amt, buffer, access, empty, full,
                                  producer_sleep, debug))


    # Start the producer and consumer threads
    for producer in producers:
        producer.start()
    for consumer in consumers:
        consumer.start()

    # Wait for the producer and consumer threads to finish
    for producer in producers:
        producer.join()
    for consumer in consumers:
        consumer.join()


    if len(buffer) == 0:
        print('\nThe buffer is empty after all the producers and consumers '
              'have finished')
    else:
        print('\nThe buffer is not empty after all the producers and consumers '
              'have finished')

    print(f'\nDone with the buffer simulation !!!')
    return

#### Running Buffer Simulation:

Now I will run the buffer simulation to see if it passes the tests, to see if
   1. We see no errors, especially pop with empty list error.
   2. The buffer never exceeds `buffer_size` (in these sims 10).
   3. Every item added to the buffer is removed at the end of each run.


To ensure that every item added to the buffer is removed at the end of each
   run, I will run the simulation multiple times I have added the lines of
   code below to the `bufferSimulation()` function to check that the buffer
   is empty once both the producer and consumer threads have finished.(`join
   ()`):
   ```python
   if len(buffer) == 0:
       print('The buffer is empty after all the producers and consumers have '
              'finished')
   else:
       print('The buffer is not empty after all the producers and consumers '
              'have finished')
   ```


##### First Buffer Simulation ( Tests 1 and 3 ):

Then to test that there are no errors when popping from the empty buffer, I
made it so that the `Producer` will sleep for 2 seconds and `Consumer` will
sleep for 1 second. This means that the `Consumer` should be able to consume
more items than the `Producer` can produce, we will see that the `buffer` never
exceeds a size of 1 because the consumer will always be taking the item in
the buffer before the producer can add another item to the buffer. If the
simulation runs with no errors and the `buffer` never exceeds a size of 1,
then the simulation passes test 1 and 3 in this simulation.

In [6]:

bufferSimulation(consumer_amt=1, producer_amt=1, consumer_sleep=1,
                     producer_sleep=3, data_amt=5, buffer_size=10, debug=True)


Producer 0 (Thread-9) added 0 to the buffer. Buffer Size: 1
Consumer 0 (Thread-8) popped 0 from the buffer. Buffer Size: 0

Producer 0 (Thread-9) added 1 to the buffer. Buffer Size: 1
Consumer 0 (Thread-8) popped 1 from the buffer. Buffer Size: 0

Producer 0 (Thread-9) added 2 to the buffer. Buffer Size: 1
Consumer 0 (Thread-8) popped 2 from the buffer. Buffer Size: 0

Producer 0 (Thread-9) added 3 to the buffer. Buffer Size: 1
Consumer 0 (Thread-8) popped 3 from the buffer. Buffer Size: 0

Producer 0 (Thread-9) added 4 to the buffer. Buffer Size: 1
Consumer 0 (Thread-8) popped 4 from the buffer. Buffer Size: 0

The buffer is empty after all the producers and consumers have finished

Done with the buffer simulation !!!


From this first simulation, we see that the buffer never exceeds a size of 1
and there are no errors when popping from the empty buffer, and the `buffer`
is empty after  the producer and consumer have finished.This means
that the simulation passes tests 1 and 3.

##### Second Buffer Simulation ( Tests 2 and 3 ):

Then to test that the buffer never exceeds the buffer size, I made it so that
the `Producer` will sleep for 0.5 seconds and `Consumer` will sleep for 3
seconds.
This means that the `Producer` will be able to produce more items than the
`Consumer` can consume, we will see that the `buffer` never exceeds a size of
10 because the `Producer` will always be adding items to the buffer before the
`Consumer` can remove items from the `buffer`. If the simulation runs with no
errors and the `buffer` never exceeds a size of 10 which is the set
`buffer_size`, then the
simulation passes.

I will run the simulation with 15 items (`data_amt = 15`) so that the `buffer`
will get full and the `Producer` will have to wait for the `Consumer` to remove
an item from the `buffer` to make room for the next item.

In [7]:

bufferSimulation(consumer_amt=1, producer_amt=1, consumer_sleep=3,
                     producer_sleep=0.5, data_amt=15, buffer_size=10,
                 debug=True)



Producer 0 (Thread-11) added 0 to the buffer. Buffer Size: 1
Consumer 0 (Thread-10) popped 0 from the buffer. Buffer Size: 0

Producer 0 (Thread-11) added 1 to the buffer. Buffer Size: 1

Producer 0 (Thread-11) added 2 to the buffer. Buffer Size: 2

Producer 0 (Thread-11) added 3 to the buffer. Buffer Size: 3

Producer 0 (Thread-11) added 4 to the buffer. Buffer Size: 4

Producer 0 (Thread-11) added 5 to the buffer. Buffer Size: 5
Consumer 0 (Thread-10) popped 1 from the buffer. Buffer Size: 4

Producer 0 (Thread-11) added 6 to the buffer. Buffer Size: 5

Producer 0 (Thread-11) added 7 to the buffer. Buffer Size: 6

Producer 0 (Thread-11) added 8 to the buffer. Buffer Size: 7

Producer 0 (Thread-11) added 9 to the buffer. Buffer Size: 8

Producer 0 (Thread-11) added 10 to the buffer. Buffer Size: 9

Producer 0 (Thread-11) added 11 to the buffer. Buffer Size: 10
Consumer 0 (Thread-10) popped 2 from the buffer. Buffer Size: 9

Producer 0 (Thread-11) added 12 to the buffer. Buffer Size: 

From this first simulation, we see that the buffer never exceeds a size of 10 `buffer_size`
 and the `buffer` is empty after  the producer and consumer have finished.
 This means that the simulation passes tests 1 and 3. We can see that initially
  the `Producer` was adding items to the buffer at a very high rate, before the
  `Consumer` was able to consume them. Then once the `buffer` was fullm with a
   size of 10, the `Producer` had to wait for the `Consumer` to remove an item
   from the `buffer`
   to make room for the next item, before the `Producer` was able to add
   another item to the `buffer`.

##### Third Buffer Simulation ( All Tests ):

Now to test all 3 tests, I made it so that the `Producer` and `Consumer` will
sleep for a random amount of time between 0.1 and 5.1 seconds and have to
process
 15 numbers of data with a `buffer_size` of 2. This means there is an
 increased chance for both an
 "overflow" and "underflow" of the buffer to occur.

In [ ]:
bufferSimulation(consumer_amt=1, producer_amt=1, consumer_sleep=-1,
                     producer_sleep=-1, data_amt=15, buffer_size=2,
                 debug=True)



Producer 0 (Thread-19) added 0 to the buffer. Buffer Size: 1
Consumer 0 (Thread-18) popped 0 from the buffer. Buffer Size: 0

Producer 0 (Thread-19) added 1 to the buffer. Buffer Size: 1

Producer 0 (Thread-19) added 2 to the buffer. Buffer Size: 2
Consumer 0 (Thread-18) popped 1 from the buffer. Buffer Size: 1
Consumer 0 (Thread-18) popped 2 from the buffer. Buffer Size: 0

Producer 0 (Thread-19) added 3 to the buffer. Buffer Size: 1
Consumer 0 (Thread-18) popped 3 from the buffer. Buffer Size: 0

Producer 0 (Thread-19) added 4 to the buffer. Buffer Size: 1
Consumer 0 (Thread-18) popped 4 from the buffer. Buffer Size: 0

Producer 0 (Thread-19) added 5 to the buffer. Buffer Size: 1
Consumer 0 (Thread-18) popped 5 from the buffer. Buffer Size: 0

Producer 0 (Thread-19) added 6 to the buffer. Buffer Size: 1
Consumer 0 (Thread-18) popped 6 from the buffer. Buffer Size: 0

Producer 0 (Thread-19) added 7 to the buffer. Buffer Size: 1

Producer 0 (Thread-19) added 8 to the buffer. Buffer Si

From this third and final simulation, we see that the `buffer` never exceeds
its `buffer_size` of 2 , there are no errors when popping from the empty, and
the `buffer` is empty after the `Producer` and `Consumer` have finished.
Therefore, the buffer simulation passes all 3 tests.

<br>

### Dining Philosophers Problem:





<br>


### Conclusion:

Overall, I found this project



#### Resources:
- [Dr. Al Madi](https://www.cs.colby.edu/nsalmadi/)
- [Using With Statements User Defined Objects](https://en.wikipedia.org/wiki/Peterson%27s_algorithm#Filter_algorithm:_Peterson's_algorithm_for_more_than_two_processes)